# Практическое задание 3 

# Классификация с использованием BERT

## курс "Математические методы анализа текстов"


### ФИО: Щербаков Василий Сергеевич

## Введение

### Постановка задачи

В этом задании вы будете классифицировать пары вопросов из stack overflow на предмет дубликатов.
Чтобы получить гораздо более высокое качество на гораздо меньшем количестве данных, чем DSSM, предлагается дообучать предобученную модель BERT.

### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
 - [Pytorch](https://pytorch.org/).
 - [Transformers](https://github.com/huggingface/transformers).
 
### Данные

Данные лежат в архиве task3_data.zip, который состоит из:

* train.tsv - обучающая выборка. В каждой строке записаны: <вопрос 1>, <вопрос 2>, <таргет>

* validation.tsv - dev выборка, которую можно использовать для подбора гиперпарамеров; например, для ранней остановки. В каждой строке через табуляцию записаны: , <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...

* test.tsv - тестовая выборка, по которой оценивается итоговое качество. В каждой строке через табуляцию записаны: , <похожий вопрос>, <отрицательный пример 1>, <отрицательный пример 2>, ...

Скачать данные можно здесь: [ссылка на google диск](https://drive.google.com/file/d/1Owb5Vpv7mVjksYo7gD9VuHkMETkzhIdr/view?usp=sharing)

## Часть 1. Подготовка данных (1 балл)

Мы будем работать с теми же данными, которые были в первом задании. А также будем учиться классифицировать пары вопросов аналогично третьей части в первом задании. Теперь выборка для обучения сгенерирована заранее :)

In [3]:
import pandas as pd
import numpy as np

import tqdm
import os
import tests

Путь к папке с данными:

In [4]:
DATA_PATH = 'data'

Считывание данных для обучения:

In [5]:
train = pd.read_table(os.path.join(DATA_PATH, 'train.tsv'))

Модель **BERT** использует специальный токенизатор Wordpiece для разбиения предложений на токены. Готовая предобученная версия такого токенизатора существует в библиотеке **transformers**. Есть два класса: **BertTokenizer** и **BertTokenizerFast**. Использовать можно любой, но второй вариант работает существенно быстрее.

Токенизаторы можно обучать с нуля на своем корпусе данных, а можно подгружать уже готовые. Готовые токенизаторы, как правило, соответствуют предобученной конфигурации модели, которая использует словарь из этого токенизатора. 

Мы будем использовать базовую конфигурацию предобученного **BERT** для модели и токенизатора:

In [6]:
BERT_MODEL = 'bert-base-uncased'

Подгружение предобученных моделей и токенизаторов в **huggingface** происходит с помощью конструктора **from_pretrained**.

В данном конструкторе можно указать либо путь к предобученному токенизатору, либо название предобученной конфигурации, как в нашем случае: тогда **transformers** сам подгрузит нужные параметры.

In [7]:
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained(BERT_MODEL)

Для классификации пар предложений необходимо привести примеры к виду: 

**[CLS] sent 1 [SEP] sent2 [SEP]**, 

где последний [SEP] можно опустить - в некоторых реализациях его используют, в некоторых нет. Существенного влияния на качество он не оказывает.

Предлагается привести все предложения из обучения к данному виду перед созданием Dataset. Для этого удобно использовать метод **tokenizer.encode_plus**, который сам вставляет специальные специальные токены [CLS], [SEP] в числовое представление примера. 

Кроме того, данный метод сразу формирует для наших примеров сегментные эмбеддинги - т.е. сопоставляет всем токенам первого предложения эмбеддинг **А**, и всем токенам второго предложения эмбеддинг **Б**.

In [8]:
def encode(query1, query2):
    """
    Args:
        query1: query text
        query2: second query text
        
    Returns:
        obj: dict {'input_ids': [0, 1, 2, 2, 1], 'token_type_ids': [0, 0, 1, 1, 1]}
    """
    return tokenizer.encode_plus(query1, query2)

tests.test_encode(encode)

In [9]:
tqdm.tqdm.pandas()

train['enc'] = train.progress_apply(lambda x: encode(x['question_1'], x['question_2']), axis=1)

c:\users\v.sherbakov\appdata\local\programs\python\python38\lib\site-packages\tqdm\std.py:706: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600000/600000 [03:00<00:00, 3325.87it/s]


Проанализируйте количество токенов в получившихся представлениях объектов, выберите максимальный порог длины, затем обрежьте все представления по этому порогу. Это необходимо для более разумного использования видеопамяти.

**hint:** можно использовать квантиль из **np.percentile**

In [10]:
lens_list = [len(token['input_ids']) for token in train['enc']]
MAXLEN = np.percentile(lens_list,95)
print(MAXLEN)
train.truncate(MAXLEN)

41.0


question_1  \
41                          Logstash split xml into array   
42      Resumable upload to google drive from JS using...   
43                           Database design for a survey   
44                  Socket IO net::ERR_CONNECTION_REFUSED   
45      How to create an HTML checkbox with a clickabl...   
...                                                   ...   
599995  animate the fill color in cylinder from bottom...   
599996              Using %f to print an integer variable   
599997          How to handle session end in global.asax?   
599998  Is there a good reason for always enclosing a ...   
599999  Cannot set the value of read-only property 'jn...   

                                               question_2  target  \
41      Excel Interop: Range.FormatConditions.Add thro...       0   
42                          Google Drive upload with CORS       1   
43      How to display data from two tables in a SQL S...       0   
44      How can I put a control in the JTableHeader of...       0   
45      How to open/create UIManagedDocument synchrono...       0   
...                                                   ...     ...   
599995                          Fill color with animation       1   
599996          using printf to print out floating values       1   
599997                      LOAD SQL Table from flat file       0   
599998    Access CKEditor iframe's style tags with jQuery       0   
599999               Google Chrome wrong character - bug?       0   

                                                enc  
41      [input_ids, token_type_ids, attention_mask]  
42      [input_ids, token_type_ids, attention_mask]  
43      [input_ids, token_type_ids, attention_mask]  
44      [input_ids, token_type_ids, attention_mask]  
45      [input_ids, token_type_ids, attention_mask]  
...                                             ...  
599995  [input_ids, token_type_ids, attention_mask]  
599996  [input_ids, token_type_ids, attention_mask]  
599997  [input_ids, token_type_ids, attention_mask]  
599998  [input_ids, token_type_ids, attention_mask]  
599999  [input_ids, token_type_ids, attention_mask]  

[599959 rows x 4 columns]

## Часть 2. Задание пайплайна обучения (2 балла)

**Внимание**. За эту часть можно получить ненулевой балл, только при демонстрации того, что ваша модель хоть как-то обучается и  работает.

### Датасет и загрузчик

Создайте датасет, из которого **DataLoader** будет брать объекты для формирования батчей.

In [74]:
from torch.utils.data import Dataset
import torch

class MyDataset(Dataset):
    
    def __init__(self, corpus, targets):
        self.__encoded = corpus['enc']
        self.__targets = targets

    def __len__(self):
        return len(self.__encoded)
    
    def __getitem__(self, idx):
        """
        Returns:
            obj: (input_ids, token_type_ids, target)
        """
        input_ids = torch.Tensor(self.__encoded[idx]['input_ids'])
        token_type_ids = torch.Tensor(self.__encoded[idx]['token_type_ids'])
        target = self.__targets[idx]
        return (input_ids, token_type_ids, target)

In [75]:
ds = MyDataset(train, train['target'])

tests.test_dataset(ds)

Реализуйте технику динамического паддинга батчей, используя функцию **collate_fn**, которую можно передать как одноименный параметр в класс **DataLoader**.

**hint**: удобно использовать метод **torch.nn.utils.rnn**. Обратите особое внимание на параметр *batch_first*

In [88]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch, pad_idx=0):
    """
        Args:
            batch: list of objects
            pad_idx: padding idx
        Returns:
            padded ids, token_type_ids, labels
    """
    input_ids, token_types, labels = zip(*batch)
    input_ids = [torch.Tensor(input_id) for input_id in input_ids]
    token_types = [torch.Tensor(token_type) for token_type in token_types]
    labels = torch.Tensor(labels)
    padded_input_ids = pad_sequence(input_ids, padding_value = pad_idx, batch_first = True) 
    padded_token_types = pad_sequence(token_types, batch_first = True) 
    res = (padded_input_ids, padded_token_types, labels)
    return res

tests.test_collator(ds, collate_fn)

In [89]:
from torch.utils.data import DataLoader

BATCH_SIZE = 50

dataloader = DataLoader(ds, collate_fn=collate_fn, batch_size=BATCH_SIZE, shuffle=True)

### Модель


В библиотеке **transformers** есть классы для модели BERT, уже настроенные под решение конкретных задач, с соответствующими головами классификации. Но гораздо более гибкий подход --- использовать энкодер BERT и, по необходимости, входной слой BERT.

Существует два способа задания модели:
* с помощью конфига **transformers.BertConfig**, в котором указываются все гиперпараметры модели
* с помощью подгрузки предобученной модели. Можно загружать как свои предобученные модели, указав путь, так и готовые предобученные модели, указав название конфигурации. В данном задании мы уже выбрали как модель базовую конфигурацию *BERT base*:

In [ ]:
from transformers import BertModel

bert = BertModel.from_pretrained(BERT_MODEL)

Напишите модель-обертку, которая:
* принимает на вход название конфигурации (или путь к предобученной модели) и загружает как свой внутренний слой, обычно называемый *backbone* слоем
* создает голову для классификации
* при вызове метода **forward** использует векторное представление токена [CLS] с последнего слоя для классификации

На вход BERT принимает:
* input_ids --- непосредственно индексы ваших токенов в словаре
* attention_mask --- булеву маску со значениями FALSE для всех PAD_IDX токенов
* token_type_ids --- индексы принадлежности токена к 1 или 2 вопросу

**hint:** в статье про BERT авторы опустили следующий архитектурный момент - представление CLS токена используется для NSP задачи, но перед классификацией оно проходит через так называемый **pooler** слой - линейный слой с *tanh* в качестве функции активации, который сохраняет размерность (т.е. на выходе оставляет hidden size значений). Если вы хотите использовать выход именно *pooler* слоя, нужно использовать вектор, получаемый из энкодера как второй элемент кортежа.

In [14]:
from torch import nn
import torch

class BERTClassifier(nn.Module):

    def __init__(self, bert, n_classes=1):
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        pass
        
    @classmethod
    def from_pretrained(cls, path, n_classes=1):
        bert = BertModel.from_pretrained(path)
        return cls(bert, n_classes)
        
    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        """
            Args:
                input_ids: token ids, shape = [batch_size, sequence_length]
                attention_mask: masks out padding tokens, shape = [batch_size, sequence_length]
                token_type_ids: segmend ids, shape = [batch_size, sequence_length]
            Returns:
                predictions, shape [batch_size]
        """
        ###########################
        ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
        ###########################
        pass

In [15]:
device = torch.device('cuda')

model = BERTClassifier.from_pretrained(BERT_MODEL, n_classes=1).to(device)

tests.test_model(dataloader, model, device)

### Оптимизатор

Для оптимизации **BERT** будем использовать **AdamW** c увеличенным learning rate'ом для параметров головы-классификатора. 

Отличие **AdamW** от **Adam** заключается в более корректной реализации $l_2$ регуляризации, которая задается параметром **weight_decay** при инициализации.

Параметры необходимо объединить на три группы:

* параметры, которым нужен weight decay --- все параметры из backbone, кроме сдвигов (bias) и LayerNorm слоев.
* остальные парамеры из backbone
* параметры головы классификации, для которых мы будем задавать гораздо больший learning rate

Будем использовать **model.named_parameters()**, чтобы разделить параметры на три группы, исходя из названий слоев.

In [16]:
NO_DECAY = ['bias', 'LayerNorm.weight']

def is_backbone(name):
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    pass

def needs_decay(name):
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    pass

def get_optimizer(model, lr, weight_decay, head_lr):
    grouped_parameters = [
        {
            'params': [param for name, param in model.named_parameters() if is_backbone(name) and needs_decay(name)],
            'lr': lr,
            'weight_decay': weight_decay,
        },
        {
            'params': [param for name, param in model.named_parameters() if is_backbone(name) and not needs_decay(name)],
            'lr': lr,
            'weight_decay': 0.,
        },
        {
            'params': [param for name, param in model.named_parameters() if not is_backbone(name)],
            'lr': head_lr,
            'weight_decay': weight_decay,
        }
    ]

    optimizer = torch.optim.AdamW(grouped_parameters, lr, weight_decay=weight_decay)

    return optimizer

LEARNING_RATE = 
WEIGHT_DECAY = 
HEAD_LEARNING_RATE = 

optimizer = get_optimizer(model, lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY, head_lr=HEAD_LEARNING_RATE)

### Scheduler


Также необходимо задать расписание для learning rate. Для **BERT** используется **linear warmup**. 

В **transformers** есть реализация **linear warmup** с помощью метода **transformers.get_linear_schedule_with_warmup**, в которой learning rate стартует с 0, и в течение **num_warmup_steps** линейно возрастает до значения, указанного в качестве стартового в оптимизаторе. Затем в течение **num_training_steps - num_warmup_steps** learning rate линейно падает до 0.

Используйте *dataloader.dataset* и *dataloader.batch_size*, чтобы рассчитать *num_training_steps* исходя из количества эпох. В случае нашей задачи одной эпохи должно быть достаточно для обучения модели.

В случае ограниченного количества видеопамяти может возникнуть ситуация, при которой батч нужного размера не влезает в видеокарту. Для таких ситуаций предлагается использовать аккумуляцию градиента - накапливание градиента в течение *accumulation_steps* с последующим шагом спуска. Т.е. делать *(loss / accumulation_steps).backward()* для каждого батча, и при этом каждые *accumulation_steps* шагов делать *optimizer.step()*.

При обучении количество шагов warmup выбирают либо как 10000 шагов, либо как 0.01% или 0.06% от всех шагов.

In [18]:
import transformers

def get_scheduler(optimizer, dataloader, n_epochs, accumulation_steps, warmup_percentage):
    ###########################
    ### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
    ###########################
    pass

N_EPOCHS = 1
ACCUMULATION_STEPS = 
WARMUP_PERCENTAGE = 

scheduler = get_scheduler(
    optimizer, dataloader, n_epochs=N_EPOCHS, accumulation_steps=ACCUMULATION_STEPS, warmup_percentage=WARMUP_PERCENTAGE
)

Для проверки качества модели необходимо использовать подготовленный для задания **Evaluator**. Важный момент: при использовании, evaluator переводит модель в режим валидации: model.eval(). Во время обучения необходимо самостоятельно переключать ее на model.train() после каждого использования.

На вход evaluator принимает вашу модель и device (CUDA или CPU), на котором необходимо считать результаты моделирования. 

При использовании evaluator можно использовать BATCH_SIZE значительно большего размера, потому что отпадает необходимость считать градиенты для параметров.

In [19]:
from utils import Evaluator

evaluator = Evaluator(os.path.join(DATA_PATH, 'validation.tsv'), tokenizer, maxlen=MAXLEN, batch_size=1024)
metrics = evaluator(model, device, verbose=True)
metrics

Данный **evaluator** предлагается использовать не только для оценки итогового качества, но также для вывода промежуточных результатов на dev сете в логи с помощью **torch.utils.tensorboard.SummaryWriter**.

Перед обучением необходимо создать объект данного класса, указав папку для записи логов.

Во время обучения через каждые $10000$ объектов необходимо записывать значения метрик в логи с помощью методов **writer.add_scalars**. Кроме того, необходимо записывать значение функционала ошибки на каждом батче во время обучения с помощью метода **writer.add_scalar**.

## Часть 3. Обучение модели (7 баллов)

Ниже предлагаются примерные значения гиперпараметров, приводящие к необходимым метрикам качества. Для подбора точных значений гиперпараметров предлагается использовать *dev set*.

**Гиперпараметры для обучения:**

* размер батча в $\{32, 64\}$
* клиппинг нормы градиента (используйте **torch.nn.utils.clip_grad_norm_**)
* шаг обучения в $\{$1e-5, 2e-5, 3e-5, 4e-5$\}$
* weight decay в $\{$1e-2, 1e-3, 1e-4$\}$
* warmup percentage в $\{0.01, 0.06\}$
* шаг обучения для головы-классификатора в $\{10, 50, 100\}$ раз больше, чем для остальных параметров

In [ ]:
from torch.utils.tensorboard import SummaryWriter

###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

In [ ]:
dev_metrics = evaluator(model, device)
dev_metrics

In [ ]:
test_evaluator = Evaluator(os.path.join(DATA_PATH, 'test.tsv'), tokenizer, maxlen=MAXLEN, batch_size=1024)
test_metrics = test_evaluator(model, device, verbose=True)
test_metrics

Задание будет засчитано на полный балл при *Hits@1* на *test set* больше $0.6$. Необходимо приложить логи из тензорборда, а также скриншот этих самых логов.

## Бонусная часть (до 6 баллов)

### ELMO-подобная архитектура для головы-классификатора (до 2 баллов)

Реализуйте и обучите ELMO-подобную архитектуру для головы-классификатора: берутся все 13 векторных представлений CLS токена, и затем с обучаемыми софтмакс-нормализуемыми весами складываются перед линейным классификатором. Дообучаются ВСЕ веса, включая сам берт. Можно попробовать зафризить исходный берт, но с большой вероятностью наибольшее качество достигается при дообучении всего.

Рекомендуется инициализировать обучаемые веса равными значениями, а также наряду с головой-классификатором присвоить им learning rate, значительно больший по значению, чем у энкодера.

Требуется получить качество хотя бы примерно такое же (а желательно и выше), чем при основной архитектуре. Может понадобиться больше эпох для обучения!


In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

### Улучшение качества на том же наборе данных (до 2 баллов)

Можно использовать любые способа для улучшения качества, КРОМЕ изменения датасетов. Например:

* Multi-sample Dropout --- при обучении, перед головой классификации итоговый вектор прогоняется через Dropout *n*-ное количество раз, каждый из полученных векторов проводится через голову классификации, и результаты усредняются.
* Изменения в архитектуре энкодера --- попробовать large конфигурацию, поменять функцию активации и прочие гиперпараметры, взять предобученный альберт из huggingface c пошаренными весами в энкодере
* попробовать дотюнить bert на MLM задачу (как в ULMFiT) перед дообучением на задачу классификации
* попробовать другие головы классификации - elmo-like голову, макс/авг пулинг по всем токенам или по всем векторам CLS токена, конкатенацию векторов CLS токена; сверточную сеть для классификации

Требуется получить Hits@1 $ \geqslant 0.65$

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

### Улучшение качества с генерацией нового тренировочного набора (до 2 баллов)

Для формирования тренировочной выборки в данном задании использовался *train_data* из первой домашней работы. Были взяты $100000$ пар дубликатов, и для первого дубликата из каждой пары также было сгенерировано 5 отрицательных примеров с помощью негативного сэмплирования. Предлагается самостоятельно сгенерировать тренировочную выборку, подобрать наилучший размер, а также количество негативных сэмплов.

Валидироваться надо на тех же самых датасетах.

Требуется получить Hits@1 $\geqslant0.7$

In [ ]:
###########################
### ╰( ͡° ͜ʖ ͡° )つ──☆*:・ﾟ
###########################

P.S.: возможна корректировка дополнительных баллов по результатам выполнения бонусов. Рекомендуется в любом случае попробовать их сделать, даже если не получится получить нужное значение метрики :)